In [6]:
from SALib.sample import saltelli
from SALib.analyze import sobol
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 定义变量及其范围
problem = {
    'num_vars': 3,
    'names': ['sugar_metabolism', 'spice_metabolism', 'vision'],
    'bounds': [[1, 5], [1, 5], [1, 10]]  # 假设代谢率范围是1到5，视野范围是1到10
}

In [9]:
from mesa.batchrunner import FixedBatchRunner  # 使用固定批量运行器
from SugarScape import SugarScapeModel

# 设置实验参数
replicates = 10
max_steps = 100
distinct_samples = 10

# 生成参数样本
param_values = saltelli.sample(problem, distinct_samples)

# 设置模型输出
model_reporters = {
    "Total Population": lambda m: m.schedule.get_agent_count()
}

# 初始化BatchRunner
batch = FixedBatchRunner(
    SugarScapeModel,
    max_steps=max_steps,
    variable_parameters={name: [] for name in problem['names']},
    model_reporters=model_reporters
)

# 初始化数据存储
count = 0
data = pd.DataFrame(index=range(replicates * len(param_values)), 
                    columns=['sugar_metabolism', 'spice_metabolism', 'vision'])
data['Run'], data['Total Population'] = None, None

# 运行实验并存储结果
for i in range(replicates):
    for vals in param_values: 
        vals = list(vals)
        # 转换为参数字典
        variable_parameters = {name: val for name, val in zip(problem['names'], vals)}

        # 运行模型迭代
        batch.run_iteration(variable_parameters, tuple(vals), count)
        iteration_data = batch.get_model_vars_dataframe().iloc[count]
        iteration_data['Run'] = count  # 确保迭代编号正确填充
        data.iloc[count, 0:3] = vals
        data.iloc[count, 3] = iteration_data['Total Population']
        count += 1

        clear_output(wait=True)
        print(f'{count / (len(param_values) * replicates) * 100:.2f}% done')

print("All experiments done!")



ImportError: cannot import name 'FixedBatchRunner' from 'mesa.batchrunner' (/opt/anaconda3/lib/python3.8/site-packages/mesa/batchrunner.py)